In [1]:
!pip install stable-baselines3

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 1.5 MB/s eta 0:00:00a 0:00:01m
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 4.7 MB/s eta 0:00:0000:0100:01
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 2.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install 'shimmy>=0.2.1'


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
# decode one hot encoded observations
import numpy as np
def one_hot_decode(vector):
    index = np.argmax(vector)
    return index

def index_to_position(index, grid_size=4):
    x = index // grid_size
    y = index % grid_size
    return x, y


one_hot_vector = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] 
index = one_hot_decode(one_hot_vector)
position = index_to_position(index, grid_size=4)  # 4x4 grid

In [28]:
from stable_baselines3 import PPO
import gym
import gym_examples
import numpy as np
import torch

env = gym.make('gym_examples/GridWorld-v0', size=4)

# create a PPO algorithm powered agent
model = PPO("MlpPolicy", env, verbose=1)
# learn with a budget of 10,000 environment interactions
model.learn(total_timesteps=10_000)

# evaluate and watch the learned policy
obs, info = env.reset()
imgs = []
for i in range(1000):
    index = one_hot_decode(one_hot_vector)
    position = index_to_position(index, grid_size=4)  # Assuming a 4x4 grid
    action, value, log_probs = model.policy.forward(torch.tensor(obs, dtype=torch.float).unsqueeze(0))
    print(f"obs: {position}, value: {float(value)}")
    
    action, states_ = model.predict(obs, deterministic=True)
    print(f"action: {type(action)}")
    obs, reward, done, truncated, info = env.step(int(action))

    # imgs += [env.render("rgb_array")] # save to video
    # VecEnv resets automatically
    if done:
        print(f"done!")
        index = one_hot_decode(one_hot_vector)
        position = index_to_position(index, grid_size=4)

        action, value, log_probs = model.policy.forward(torch.tensor(obs, dtype=torch.float).unsqueeze(0))

        # print(f"final obs: {position}, final value: {float(value)}")
        obs, info = env.reset()
env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/Users/andyyang/Desktop/DS598 Reinforcement Learning/gym-examples/.env/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/Users/andyyang/Desktop/DS598 Reinforcement Learning/gym-examples/.env/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 283      |
|    ep_rew_mean     | 10       |
| time/              |          |
|    fps             | 6263     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 146         |
|    ep_rew_mean          | 10          |
| time/                   |             |
|    fps                  | 4276        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007727754 |
|    clip_fraction        | 0.0547      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.0129      |
|    learning_rate        | 0.

In [26]:
import torch
obs, info = env.reset()

# Convert the observation to a PyTorch tensor and add a batch dimension
obs_tensor = torch.tensor(obs, dtype=torch.float).unsqueeze(0)  # Add batch dimension

# Assuming `model` is your trained PPO model
# Note: Adjust based on SB3's version and specific model architecture
actions, values, log_probs = model.policy.forward(obs_tensor)

print("Estimated values:", float(values))


Estimated values: 7.3430891036987305
